In [39]:
import sys
import datetime
import time
import re
import os

In [40]:
# To add path so that we can import zeno_etl_libs from local folder
sys.path.append('../../../..')

In [41]:
from zeno_etl_libs.helper.run_notebook import run_notebook
from zeno_etl_libs.helper.parameter.job_parameter import parameter

In [42]:
env = "prod"
notebook_file = "\\scripts\\ipc2-ss-main\\ipc2_ss_main.ipynb"
parameters = {
    "env": "prod",
    "email_to":"vivek.revi@zeno.health", 
    "debug_mode":"N"
}

In [43]:
os.environ['env'] = 'dev'

In [47]:
args = parameter.get_params(job_id=140)
reset_stores = args["reset_stores"]
batch_size = 1
store_batch_split = [reset_stores[i:i+batch_size] for i in range(0, len(reset_stores), batch_size)]

Redshift DB connection closed successfully.


In [48]:
store_batch_split

[[23], [31]]

In [49]:
os.environ['env'] = env

In [50]:
base = "\\".join(os.getcwd().split("\\")[:-2])

In [51]:
input_file = base + notebook_file

In [52]:
output_path = base + "\\run\\logs"

In [53]:
name = os.path.basename(notebook_file)
nb_name, nb_ext = os.path.splitext(name)
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
result = "{}-{}{}".format(nb_name, timestamp, nb_ext)

In [54]:
for batch_stores in store_batch_split:
    run_batch = store_batch_split.index(batch_stores) + 1
    tot_batch = len(store_batch_split)
    
    # add to parameters
    parameters["run_batch"] = run_batch
    parameters["tot_batch"] = tot_batch
    parameters["batch_stores"] = batch_stores
    
    if env == "dev":
        import papermill as pm
        pm.execute_notebook(
            input_path=input_file,
            output_path=f"{output_path}/{result}",
            parameters=parameters
        )
    elif env in ("stage", "prod"):
        run_notebook.run_notebook(
            image=f"{env}-notebook-runner",
            notebook=input_file,
            output=output_path,
            parameters=parameters,
            timeout_in_sec=3600,
            in_vpc=True,
            instance_type="ml.c5.4xlarge",
            env=env,
            check_completion_status=False
        )

Job prod-ipc2-ss-main-2023-02-09-08-51-17 started
Job prod-ipc2-ss-main-2023-02-09-08-51-19 started


In [ ]:
# instances: 
# ml.c5.2xlarge, ml.c5.4xlarge, ml.c5.9xlarge